# Ungraded Lab: Using Callbacks to Control Training

In this lab, you will use the [Callbacks API](https://keras.io/api/callbacks/) to stop training when a specified metric is met. This is a useful feature so you won't need to complete all epochs when this threshold is reached. For example, if you set 1000 epochs and your desired accuracy is already reached at epoch 200, then the training will automatically stop. Let's see how this is implemented in the next sections.


## Load and Normalize the Fashion MNIST dataset

Like the previous lab, you will use the Fashion MNIST dataset again for this exercise. And also as mentioned before, you will normalize the pixel values to help optimize the training.

In [1]:
import tensorflow as tf
import keras


In [2]:

# Instantiate the dataset API

fmnist = tf.keras.datasets.fashion_mnist

# Load the dataset
(x_train, y_train), (x_test, y_test) = fmnist.load_data()

# Normalize the pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

In [11]:
len(x_train) / 32

1875.0

## Creating a Callback class

You can create a callback by defining a class that inherits the [tf.keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback) base class. From there, you can define available methods to set where the callback will be executed. For instance below, you will use the [on_epoch_end()](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback#on_epoch_end) method to check the loss at each training epoch.

In [41]:
class myCallback(keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.count = 0
        
    def on_epoch_end(self, epoch, logs=None):
        '''
        Halts the training when the loss falls below 0.4

        Args:
            epoch (integer) - index of epoch (required but unused in the function definition below)
            logs (dict) - metric results from the training epoch
        '''

        # Check the loss
        if logs['loss'] < 0.4:

            # Stop if threshold is met
            print("\nLoss is lower than 0.4 so cancelling training!")
            self.model.stop_training = True
            
    def on_train_begin(self, logs=None):
        print(f'\non_train_begin ... ...')
        print(f'\nmodel name: {self.model.layers}')
        
        return super().on_train_begin(logs)
    
    def on_train_batch_begin(self, batch, logs=None):
        #print(f'\non_train_batch_begin ... ...')
        self.count += 1
        return super().on_train_batch_begin(batch, logs)
    
    def on_train_batch_end(self, batch, logs=None):
        if self.count % 1875 == 0:
            print(f'\non_train_batch_end: {self.count}')
        return super().on_train_batch_end(batch, logs)
    
    def on_batch_end(self, batch, logs=None):
        #print(f'\non_batch_end')
        return super().on_batch_end(batch, logs)
    
    def on_train_end(self, logs=None):
        return super().on_train_end(logs)

## Define and compile the model

Next, you will define and compile the model. The architecture will be similar to the one you built in the previous lab. Afterwards, you will set the optimizer, loss, and metrics that you will use for training.

In [39]:
# Define the model
model = keras.models.Sequential([
    keras.Input(shape=(28,28)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Compile the model
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train the model

Now you are ready to train the model. To set the callback, simply set the `callbacks` parameter to an instance of `myCallback` put into a list. Run the cell below and observe what happens.

In [42]:
# Train the model with a callback
model.fit(x_train, y_train, epochs=10, callbacks=[myCallback()], verbose=2)


on_train_begin ... ...

model name: [<Flatten name=flatten_8, built=True>, <Dense name=dense_16, built=True>, <Dense name=dense_17, built=True>]
Epoch 1/10

on_train_batch_end: 1875

Loss is lower than 0.4 so cancelling training!
1875/1875 - 11s - 6ms/step - accuracy: 0.8820 - loss: 0.3200


: 

You will notice that the training does not need to complete all 10 epochs. By having a callback at the end of each epoch, it is able to check the training parameters and compare if it meets the threshold you set in the function definition. In this case, it will simply stop when the loss falls below `0.40` after the current epoch.

*Optional Challenge: Modify the code to make the training stop when the accuracy metric exceeds 60%.*

That concludes this simple exercise on callbacks!